<a href="https://colab.research.google.com/github/ngolla/video-captioning/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pickle
from google.colab import drive
from pathlib import Path
import numpy as np

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
drive_path=Path('/content/drive/My Drive')
list(drive_path.glob('*'))

[PosixPath('/content/drive/My Drive/Colab Notebooks'),
 PosixPath('/content/drive/My Drive/YouTubeClips.tar'),
 PosixPath('/content/drive/My Drive/video_corpus.csv'),
 PosixPath('/content/drive/My Drive/.ipynb_checkpoints'),
 PosixPath('/content/drive/My Drive/VideoArrays'),
 PosixPath('/content/drive/My Drive/pickle_train.dat'),
 PosixPath('/content/drive/My Drive/pickle_test.dat')]

In [5]:
train_file=drive_path.joinpath('pickle_train.dat')
val_file=drive_path.joinpath('pickle_test.dat')

In [6]:
(vid_test, cap_test)=pickle.load(open( val_file, "rb" ))
(vid_train, cap_train)=pickle.load(open( train_file, "rb" ))

In [7]:
print(len(vid_train), vid_train[0].shape)
print(len(vid_test), vid_test[0].shape)

64639 (80, 4096)
16160 (80, 4096)


In [8]:
vid=np.array(vid_test[:500])
cap=np.array(cap_test[:500])

In [9]:
print(vid.ndim, vid.shape )
print(cap.ndim, cap.shape)

3 (500, 80, 4096)
2 (500, 89)


In [10]:
vid.shape[1], vid.shape[2]

(80, 4096)

In [11]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, LSTM, Dense

# returns train, inference_encoder and inference_decoder models
def basic_enc_dec(n_input, n_output, n_units):
    # define training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]

    # define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)

    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

    # return all models
    return model, encoder_model, decoder_model

In [14]:
vocab_size=11161
dim_embedding=64
maxlen=80

In [15]:
model, enc, dec = basic_enc_dec(4096, vocab_size, maxlen)

In [18]:
cap

array([  2,  25,   4, 469, 528,   3,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [19]:

x2 = np.hstack([np.zeros((500, 1)), np.array(cap)])
x2 = x2[:, :-1]

In [25]:
#Convert to 1652x42x1000
from keras.utils.np_utils import to_categorical   

x2_in = to_categorical(x2, num_classes = vocab_size)
outputs = to_categorical(cap, num_classes = vocab_size)
print(x2_in.shape, outputs.shape)

(500, 89, 11161) (500, 89, 11161)


In [ ]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=7e-4), loss='categorical_crossentropy')
history=model.fit([vid, x2_in], outputs, epochs = 1)

In [29]:
history.history

{'loss': [4.78618049621582]}

In [16]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 4096)] 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 11161) 0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 80), (None,  1336640     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 80), ( 3597440     input_2[0][0]                    
                                                                 lstm[0][1]            

In [66]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Input,LSTM, Embedding, RepeatVector, Bidirectional, BatchNormalization, Dropout, TimeDistributed, Activation
from keras.models import Model
from keras import layers

In [ ]:
# Model

image_model = Sequential()
image_model.add(Dense(EMBEDDING_DIM, input_shape=(4096,), activation='relu'))
image_model.add(RepeatVector(max_length))

   
lang_model = Sequential()
lang_model.add(Embedding(vocab_size,EMBEDDING_DIM , input_length=max_length))
lang_model.add(Bidirectional(LSTM(256,return_sequences=True)))
lang_model.add(Dropout(0.5))
lang_model.add(BatchNormalization())
lang_model.add(TimeDistributed(Dense(EMBEDDING_DIM)))

   
fin_model = Sequential()
fin_model.add(Merge([image_model, lang_model], mode='concat'))
    #model.add(Concatenate([image_model, lang_model]))
fin_model.add(Dropout(0.5))
fin_model.add(BatchNormalization())
fin_model.add(Bidirectional(LSTM(1000,return_sequences=False)))

fin_model.add(Dense(vocab_size))
fin_model.add(Activation('softmax'))
print ("Model created!")
    


In [94]:
input_image = Input(shape=(vid.shape[1],vid.shape[2]), batch_size=100)
fimage = Dense(256,activation='relu',name="ImageFeature")(input_image)

In [95]:
input_txt = Input(shape=(maxlen,))
ftxt = Embedding(vocab_size,dim_embedding, mask_zero=True)(input_txt)
ftxt = LSTM(256,name="CaptionFeature",return_sequences=True)(ftxt)

In [96]:
decoder = layers.add([ftxt,fimage])
decoder = Dense(256,activation='relu')(decoder)
output = Dense(vocab_size,activation='softmax')(decoder)
model = Model(inputs=[input_image, input_txt],outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam')

print(model.summary())

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 80)]         0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 80, 64)       714304      input_17[0][0]                   
__________________________________________________________________________________________________
input_16 (InputLayer)           [(100, 80, 4096)]    0                                            
__________________________________________________________________________________________________
CaptionFeature (LSTM)           (None, 80, 256)      328704      embedding_7[0][0]                
_______________________________________________________________________________________

In [93]:
cap

array([  2,  25,   4, 469, 528,   3,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [97]:
model.fit(vid,cap, batch_size=64, epochs=1)

AssertionError: ignored

In [86]:
print(input_txt.shape)
print(ftxt.shape)

(None, 80)
(None, 80, 256)


In [83]:
print(input_image.shape)
print(fimage.shape)

(64, 80, 4096)
(64, 80, 256)


TensorShape([64, 80, 256])

In [24]:
model=Sequential()
model.add(Input(batch_size=100,shape=(vid.shape[1],vid.shape[2])))

In [33]:
lstm = LSTM(256)
output = lstm(vid)
print(output.shape)

(500, 89)


In [32]:
output

<tf.Tensor: shape=(500, 4), dtype=float32, numpy=
array([[ 1.90382108e-01, -3.94081295e-01,  2.11420476e-01,
         1.04973800e-01],
       [ 2.19033137e-01, -5.93804896e-01,  5.73776588e-02,
        -4.32912469e-01],
       [ 3.64056975e-01, -4.21474189e-01,  3.83645773e-01,
        -1.68706283e-01],
       ...,
       [ 6.56669259e-01,  1.22925855e-01,  2.14053512e-01,
         3.43979746e-01],
       [ 1.29683375e-01,  1.46441936e-01,  1.90202013e-01,
        -4.43249708e-04],
       [-3.48755777e-01, -4.98347543e-02,  3.26643020e-01,
        -1.47544527e-02]], dtype=float32)>